# Bayes Course Pre-Work

As a first step, import some of the libraries we will be using in the course. This will test that everything is installed correctly on your system. Running the cell below should not return errors.

In [1]:
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
import pandas as pd
import pymc as pm
import sklearn as sk
import seaborn as sns
import pytensor.tensor as pt
import pytensor

print('All packages imported successfully!')

All packages imported successfully!


If everything was imported, you are ready to work through the exercises below.

## Data Import and Manipulation

Loading and processing data has become easier since the development of the `pandas` library, which provides data structures and functions for automating key data operations. 

For importing data from most common storage formats, several `read_*` functions are available.

In [2]:
[f for f in dir(pd) if f.startswith('read_')]

['read_clipboard',
 'read_csv',
 'read_excel',
 'read_feather',
 'read_fwf',
 'read_gbq',
 'read_hdf',
 'read_html',
 'read_json',
 'read_orc',
 'read_parquet',
 'read_pickle',
 'read_sas',
 'read_spss',
 'read_sql',
 'read_sql_query',
 'read_sql_table',
 'read_stata',
 'read_table',
 'read_xml']

The `../data/` subdirectory includes some clinical trial data in `cdystonia.csv`. Choose the appropriate function and use it to import this data to a variable called `cdystonia`. Print the first 15 lines of the resulting `DataFrame`.

In [3]:
cdystonia = pd.read_csv("../data/cdystonia.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../data/cdystonia.csv'

This dataset is from [Statistical Methods for the Analysis of Repeated Measurements](http://www.amazon.com/Statistical-Methods-Analysis-Repeated-Measurements/dp/0387953701) by Charles S. Davis, pp. 161-163 (Springer, 2002). These data are from a multicenter, randomized controlled trial of botulinum toxin type B (BotB) in patients with cervical dystonia from nine U.S. sites.

Patients were randomized to placebo (N=36), 5000 units of BotB (N=36), or 10,000 units of BotB (N=37). The response variable is the total score on Toronto Western Spasmodic Torticollis Rating Scale (TWSTRS), measuring severity, pain, and disability of cervical dystonia (high scores mean more impairment). TWSTRS was measured at baseline (week 0) and weeks 2, 4, 8, 12, 16 after treatment began, so this is a longitudinal study.

1. Since there are repeated measures of each patient, the `patient` column alone cannot be used as an index, because it is not unique. Use some of the columns in the DataFrame to create an index for the data that is unique.

In [ ]:
# Your answer here

2. The `cdystonia` dataset is stored in **long** format, meaning that each row contains a single observation. Use pandas functions and methods to change the data to **wide** format, where each row represents the data for a single patient. 

In [ ]:
# Your answer here

3. To get an idea about the efficacy of the treatment, use `pandas` to group the data by treatment group, and calculate the mean and standard deviation of the `twstrs` outcome variable for each group in week 4.

In [ ]:
# Your answer here
# As a sanity check, this should be the answer:

#                mean       std
# treat
# 10000U         34.805556  12.188565
# 5000U          37.114286  15.311993
# Placebo        39.342857  11.827045

4. Since this is a longitudinal study, graphics are helpful for understanding the dynamics of the experiment. Using the plotting package of your choice (there are many for Python!) create a set of plots showing how the response variable changes over time for each experimental group. 

In [ ]:
# Your answer here

## Linear Algebra


Operations become much faster when we can express them as vectorized linear algebra commands. One example is generating multivariate normal distributions.

Suppose we wish to generate samples with mean $\mu$ and covariance $\Sigma$, where $\mu$ is $n \times 1$ and $\Sigma$ is $n \times n$. One way to do this is to calculate a [*Cholesky decomposition*](https://en.wikipedia.org/wiki/Cholesky_decomposition) of $\Sigma$, so that
$$
\Sigma = LL^T.
$$

Then if $x$ is $n$ independent draws from a standard normal distribution, 
$$
\nu = Lx + \mu \sim \mathcal{N}(\mu, \Sigma)
$$

### Exercises
Let $\mu = (3, 2)$, and
$$
\Sigma = \left(\begin{array}{cc}
1 & 0.9 \\
0.9 & 1
\end{array}\right)
$$

1. Use `np.linalg.cholesky` to compute $L$, the Cholesky decomposition of $\Sigma$
2. In newer Python (3.6+), `@` is matrix multiplication. Confirm that $\Sigma = LL^T$.
3. Draw 2 independent draws from a standard normal using `x = np.random.randn(2)`, and compute $\nu$ using `L @ x + mu`
4. The above exercise generates 1 draw from a multivariate normal. Use `x = np.random.randn(2, 1_000)` and the same formula as above to generate 1,000 draws.

In [ ]:
mu = np.array([[3], [2]])
sigma = np.array([[1, 0.9], [0.9, 1]])

In [ ]:
# Use `np.linalg.cholesky` to compute $L$, the Cholesky decomposition of `sigma`

In [ ]:
# In newer Python (3.7+), `@` is matrix multiplication. Confirm that `sigma = L @ L.T`

In [ ]:
# Draw 2 independent draws from a standard normal using `x = np.random.randn(2)`, and compute `nu` using `L @ x + mu`

In [ ]:
# The above exercise generates 1 draw from a multivariate normal. Use `x = np.random.randn(2, 1_000)` and the same formula as above to generate 1,000 draws

## Probability Distributions and Simulation

Bayesian inference relies on the use of probability distributions for constructing models. Though several statistical and machine learning packages implement their own set of probability distributions, the NumPy and SciPy libraries include general-purpose functions and classes for performing probability operations. 

NumPy has an efficient set of random number generators for different distributions, while SciPy implements a large set of complete probability distributions that allow them to be used in most applications.

In [ ]:
from scipy.stats import distributions as dists

In [ ]:
dir(dists)[-20:]

Let's look at a gamma distribution as an example:

In [ ]:
gamma_dist = dists.gamma

Inspecting the attributes of `gamma_dist`, we can see several important methods, including `pdf` (probability distribution function), `cdf` (cumulative distribution function), `rvs` (random number generator), and more.

In [ ]:
[f for f in dir(gamma_dist) if not f.startswith('_')]

### Exercise: Simulating linear regression

1. Generate a (1000, 10) `features` array using a uniform distribution.
2. Generate a (10, 1) `weights` array, using a normal distribution with standard deviation 2 and mean 0. Note this is typically unobserved.
3. Generate a (1000, 1) `noise` array, using a normal distribution with standard deviation 1 and mean 0. Note that this is typically unobserved.
4. Compute (1000, 1) `target` array, as `features @ weights + noise`
5. Recover an estimate for the weights using `np.linalg.pinv(features) @ target` (`pinv` is the [Moore-Penrose pseudoinverse](https://en.wikipedia.org/wiki/Moore%E2%80%93Penrose_inverse))
6. Our statistical model is 
$$
\text{weights} \sim \mathcal{N}(0, 2) \\
\text{target} | \text{features}, \text{weights} \sim \mathcal{N}(\text{features} \cdot \text{weights}, 1)
$$
What is the log probability that all the weights are 0, and all the targets are 1, given your generated `features`?
7. What is the log probability of your generated `weights` and generated `targets`, given your generated `features`?

In [ ]:
# Generate a (1000, 10) `features` array using a uniform distribution.

# assert features.shape == (1000, 10)

In [ ]:
# Generate a (10, 1) `weights` array, using a normal distribution with standard deviation 2 and mean 0

# assert weights.shape == (10, 1)

In [ ]:
# Generate a (1000, 1) `noise` array, using a normal distribution with standard deviation 1 and mean 0

# assert noise.shape == (1000, 1)

In [ ]:
# Compute (1000, 1) `target` array, as `features @ weights + noise`

# assert target.shape == (1000, 1)

In [ ]:
# Recover an estimate for the weights using `np.linalg.pinv(features) @ target`

In [ ]:
# Compute the log probability that all the weights are 0, and all the targets are 1, given your generated `features`

In [ ]:
# Compute the log probability of your generated `weights` and `targets`, given your generated `features`

Your answer to 5 above should match the answer `scikit-learn` provides:

In [ ]:
# from sklearn.linear_model import LinearRegression

# LinearRegression(fit_intercept=False).fit(features, target).coef_.T

## Optimization

Another way of running linear regression is via optimization. Suppose we want to minimize the sum of squares using scipy. We can do this using `scipy.optimize`.

In [ ]:
from scipy.optimize import fmin_bfgs

features = np.random.uniform(size=(1000, 10))
weights = np.random.normal(0, 2, size=(10, 1))
noise = np.random.normal(0, 1, size=(1000, 1))
target = (features @ weights + noise).flatten()


### Exercise

Find the weights with the least mean squared error using `fmin_bfgs`. It expects a function to minimize and an initial point.

1. The function to minimize should accept an argument `x`, and return `((features @ x - target) ** 2).mean()`
2. You can initialize with a vector of 10 zeros.

In [ ]:
# Write the function to minimize, and an initial point

def loss_function(x):
    pass

# initial_point = 

# assert initial_point.shape == (10,)

In [ ]:
# min_x = fmin_bfgs(loss_function, initial_point)
# min_x

This answer should match our solution using linear algebra, or using scikit-learn:

In [ ]:
np.linalg.pinv(features) @ target

In [ ]:
from sklearn.linear_model import LinearRegression

LinearRegression(fit_intercept=False).fit(features, target).coef_.T